In [1]:

import random
import math
import time
from collections import Counter

#Itens da lista
itens = [
    (60, 10),  # (valor, peso)
    (100, 20),
    (120, 30),
    (90, 15),
    (30, 5),
    (70, 12),
    (40, 7),
    (160, 25),
    (20, 3),
    (50, 9),
    (110, 18),
    (85, 14),
    (95, 16),
    (200, 28),
    (55, 6)
]

#define a constante com o valor 50, que é a capacidade máxima do caminhão.
MAX_WEIGHT = 50

#função fitness
def fitness(lista):
    """Calcula o valor e o peso para ver se o total respeita o peso máximo."""
    total_value, total_weight = 0, 0 
     
    for i, bit in enumerate(lista):
        if bit == 1:
            total_value += itens[i][0] 
            total_weight += itens[i][1] 
    if total_weight > MAX_WEIGHT: 
        return 0  
    return total_value 

def peso(lista):
    """Retorna o peso total da solução."""
    return sum(itens[i][1] for i, bit in enumerate(lista) if bit == 1)

def teste():
    """Cria uma solução aleatória e repara se necessário."""
    verificacao = [random.randint(0, 1) for _ in itens]
    while peso(verificacao) > MAX_WEIGHT:
        foi = [i for i, bit in enumerate(verificacao) if bit == 1]
        if foi:
            verificacao[random.choice(foi)] = 0
    return verificacao 

def vizinho(lista):
    """Gera um vizinho alterando 1 item (flip)."""
    i = random.randrange(len(lista))
    nova_verificacao = lista[:]
    nova_verificacao[i] = 1 - nova_verificacao[i]
    while peso(nova_verificacao) > MAX_WEIGHT:
        foi = [j for j, bit in enumerate(nova_verificacao) if bit == 1]
        if foi:
            nova_verificacao[random.choice(foi)] = 0 
    return nova_verificacao 

#Hill Climbing
def hill_climbing(max_inter=300):
    solucao = teste() 
    valor_otimo = fitness(solucao)
    
    for _ in range(max_inter):
        invertido = vizinho(solucao)
        if fitness(invertido) > valor_otimo:
            solucao = invertido 
            valor_otimo = fitness(invertido)
    
    return solucao, fitness(solucao), peso(solucao)

    

#Simulated Annealing
def simulated_annealing(T0=50.0, Tmin=0.1, alpha=0.95, passos_por_T=30):
    solucao = teste()
    solucao2 = solucao[:]
    T = T0
    
    while T > Tmin:
        for _ in range(passos_por_T):
            invertido = vizinho(solucao)
            delta = fitness(invertido) - fitness(solucao)
            if delta > 0 or random.random() < math.exp(delta / T):
                solucao = invertido
                if fitness(invertido) > fitness(solucao2):
                    solucao2 = invertido
        T *= alpha
    
    return solucao2, fitness(solucao2), peso(solucao2)

#Algoritmo Genético
def selecao_torneio(pai, k=3):
    selected = random.sample(pai, k)
    return max(selected, key=lambda ind: fitness(ind))

def crossover(parente1, parente2):
    if random.random() < 0.9:
        ponto = random.randint(1, len(parente1)-1)
        filho1 = parente1[:ponto] + parente2[ponto:]
        filho2 = parente2[:ponto] + parente1[ponto:]
    else:
        filho1, filho2 = parente1[:], parente2[:]
    return filho1, filho2

def mutacao(individual, p_mut=0.02):
    for i in range(len(individual)):
        if random.random() < p_mut:
            individual[i] = 1 - individual[i]
    while peso(individual) > MAX_WEIGHT:
        foi = [j for j, bit in enumerate(individual) if bit == 1]
        if foi:
            individual[random.choice(foi)] = 0
    return individual

def algoritmo_genetico(pop_size=50, generations=120):
    populacao = [teste() for _ in range(pop_size)]#
    
    for _ in range(generations):
        nova_populacao = []
        
        # Elitismo: mantém os 2 melhores
        elites = sorted(populacao, key=lambda ind: fitness(ind), reverse=True)[:2]
        nova_populacao.extend(elites)
        
        # Geração de novos indivíduos
        while len(nova_populacao) < pop_size:
            p1 = selecao_torneio(populacao)
            p2 = selecao_torneio(populacao)
            c1, c2 = crossover(p1, p2)
            c1 = mutacao(c1)
            c2 = mutacao(c2)
            nova_populacao.extend([c1, c2])
        
        populacao = nova_populacao[:pop_size]

    solucao2 = max(populacao, key=lambda ind: fitness(ind))
    return solucao2, fitness(solucao2), peso(solucao2)

#Monte Carlo (100 execuções)
def monte_carlo(alg_func, n_exec=100):
    resultados = []
    for _ in range(n_exec):
        verificacao, val, w = alg_func()
        resultados.append((tuple(verificacao), val, w))
    return resultados

def analisar(resultados):
    valores = [r[1] for r in resultados]
    solucoes = [r[0] for r in resultados]

    media = sum(valores) / len(valores)
    melhor = max(valores)
    distintas = len(set(solucoes))
    frequencia = Counter(solucoes).most_common(3)

    return media, melhor, distintas, frequencia

# Execução e comparação
if __name__ == "__main__":

    print("=== Monte Carlo: Hill Climbing ===")
    hc_resultados = monte_carlo(hill_climbing)
    hc_media, hc_melhor, hc_distintas, hc_freq = analisar(hc_resultados)
    print(f"Valor médio (consistência): {hc_media:.2f}")
    print(f"Melhor valor obtido (resultado global): {hc_melhor}")
    print(f"Soluções distintas (diversidade): {hc_distintas}")
    print("Soluções mais frequentes:", hc_freq, "\n")

    print("=== Monte Carlo: Simulated Annealing ===")
    sa_resultados = monte_carlo(simulated_annealing)
    sa_media, sa_melhor, sa_distintas, sa_freq = analisar(sa_resultados)
    print(f"Valor médio (consistência): {sa_media:.2f}")
    print(f"Melhor valor obtido (resultado global): {sa_melhor}")
    print(f"Soluções distintas (diversidade): {sa_distintas}")
    print("Soluções mais frequentes:", sa_freq, "\n")

    print("=== Monte Carlo: Algoritmo Genético ===")
    ag_resultados = monte_carlo(algoritmo_genetico)
    ag_media, ag_melhor, ag_distintas, ag_freq = analisar(ag_resultados)
    print(f"Valor médio (consistência): {ag_media:.2f}")
    print(f"Melhor valor obtido (resultado global): {ag_melhor}")
    print(f"Soluções distintas (diversidade): {ag_distintas}")
    print("Soluções mais frequentes:", ag_freq, "\n")

    #Análise final
    print("=== Análise Final ===")

    #Melhor resultado global
    melhor_global = max(
        [("Hill Climbing", hc_melhor),
         ("Simulated Annealing", sa_melhor),
         ("Algoritmo Genético", ag_melhor)],
        key=lambda x: x[1]
    )
    print(f"a) Melhor resultado global: {melhor_global[0]} com valor {melhor_global[1]}")

    #Mais consistente (menor variação)
    variacoes = {
        "Hill Climbing": max(r[1] for r in hc_resultados) - min(r[1] for r in hc_resultados),
        "Simulated Annealing": max(r[1] for r in sa_resultados) - min(r[1] for r in sa_resultados),
        "Algoritmo Genético": max(r[1] for r in ag_resultados) - min(r[1] for r in ag_resultados),
    }
    mais_consistente = min(variacoes.items(), key=lambda x: x[1])
    print(f"b) Mais consistente: {mais_consistente[0]} (variação {mais_consistente[1]})")

    #Maior diversidade
    diversidade = {
        "Hill Climbing": hc_distintas,
        "Simulated Annealing": sa_distintas,
        "Algoritmo Genético": ag_distintas,
    }
    mais_diverso = max(diversidade.items(), key=lambda x: x[1])
    print(f"c) Maior diversidade: {mais_diverso[0]} ({mais_diverso[1]} soluções distintas)")

=== Monte Carlo: Hill Climbing ===
Valor médio (consistência): 339.55
Melhor valor obtido (resultado global): 350
Soluções distintas (diversidade): 11
Soluções mais frequentes: [((0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1), 48), ((0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0), 28), ((0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1), 9)] 

=== Monte Carlo: Simulated Annealing ===
Valor médio (consistência): 349.95
Melhor valor obtido (resultado global): 350
Soluções distintas (diversidade): 2
Soluções mais frequentes: [((0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1), 99), ((0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1), 1)] 

=== Monte Carlo: Algoritmo Genético ===
Valor médio (consistência): 350.00
Melhor valor obtido (resultado global): 350
Soluções distintas (diversidade): 1
Soluções mais frequentes: [((0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1), 100)] 

=== Análise Final ===
a) Melhor resultado global: Hill Climbing com valor 350
b) Mais consistente: Algoritmo Genético (variação 0)
c) 